### Imported the required packages

In [1]:
import pennylane as qml
import numpy as np
import torch
from torch.autograd import Variable
np.random.seed(42)


C:\Users\Bhargav\anaconda3\envs\Qiskit_1\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Bhargav\anaconda3\envs\Qiskit_1\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
C:\Users\Bhargav\anaconda3\envs\Qiskit_1\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


### Constructed the variational circuit

In [2]:
# number of qubits in the circuit
nr_qubits = 4
# number of layers in the circuit
nr_layers = 2

# randomly initialize parameters from a normal distribution
params = np.random.normal(0, np.pi, (nr_qubits*4, nr_layers, 4))
params = Variable(torch.tensor(params), requires_grad=True)

# a layer of the circuit ansatz
def layer(params, j):
    
    for i in range(nr_qubits):
        qml.RX(params[i, j, 0], wires=i)
        qml.RY(params[i, j, 1], wires=i)
        qml.RZ(params[i, j, 2], wires=i)
        qml.RZ(params[i, j, 3], wires=i)
        
    qml.CNOT(wires=[0, 1])
    qml.CNOT(wires=[0, 2])
    qml.CNOT(wires=[0, 3])
    qml.CNOT(wires=[1, 2])
    qml.CNOT(wires=[1, 3])
    qml.CNOT(wires=[2, 3])

    


In [3]:
params

tensor([[[ 1.5605, -0.4344,  2.0348,  4.7847],
         [-0.7356, -0.7356,  4.9612,  2.4110]],

        [[-1.4749,  1.7045, -1.4559, -1.4631],
         [ 0.7601, -6.0107, -5.4190, -1.7665]],

        [[-3.1819,  0.9872, -2.8526, -4.4369],
         [ 4.6045, -0.7093,  0.2121, -4.4760]],

        [[-1.7102,  0.3485, -3.6160,  1.1803],
         [-1.8870, -0.9164, -1.8903,  5.8191]],

        [[-0.0424, -3.3229,  2.5841, -3.8354],
         [ 0.6562, -6.1565, -4.1726,  0.6185]],

        [[ 2.3200,  0.5384, -0.3633, -0.9459],
         [-4.6449, -2.2615, -1.4471,  3.3210]],

        [[ 1.0795, -5.5388,  1.0181, -1.2098],
         [-2.1266,  1.9216,  3.2390,  2.9257]],

        [[-2.6365, -0.9714,  1.0407,  3.0648],
         [-1.5054, -0.5833, -3.4757, -3.7580]],

        [[ 2.5526,  4.2608, -0.2262,  3.1527],
         [ 1.1361, -2.0267,  1.1354,  4.8319]],

        [[-0.1126,  4.9155, -8.2302,  2.5821],
         [ 0.2735, -0.9394,  0.2883, -6.2441]],

        [[-0.6901,  1.1219,  4.6429, -1.

### We use the "default.qubit" device to perform the optimization

In [4]:
dev = qml.device("default.qubit", wires=4)


In [5]:
@qml.qnode(dev, interface="torch")
def circuit(params, A,input_1):
    
   
    
    
    qml.QubitStateVector(input_1, wires=[0,1,2,3])
        
  
        
        
    
    # repeatedly apply each layer in the circuit
    for j in range(nr_layers):
        layer(params, j)
    
    # measure the expectation value for each of the 4 qubits
    a=qml.expval(qml.Hermitian(A, wires=0))
    b=qml.expval(qml.Hermitian(A, wires=1))
    c=qml.expval(qml.Hermitian(A, wires=2))
    d=qml.expval(qml.Hermitian(A, wires=3))
    

    
     
    return a,b,c,d

### Prepare 4 random state-vectors (on 4 qubits)

In [6]:
from qiskit import quantum_info as qf
rand_sv_1=np.array(qf.random_statevector(16)) # Statevector correspondong to first 4-qubit random state
rand_sv_2=np.array(qf.random_statevector(16)) # Statevector correspondong to second 4-qubit random state
rand_sv_3=np.array(qf.random_statevector(16)) # Statevector correspondong to third 4-qubit random state
rand_sv_4=np.array(qf.random_statevector(16)) # Statevector correspondong to fourth 4-qubit random state

rand_sv_f= np.array([rand_sv_1,rand_sv_2,rand_sv_3,rand_sv_4]) # Put all of them in a single array


rand_sv_f

array([[-0.0957305 +0.07856306j,  0.04760377+0.1092431j ,
         0.19821649-0.40359278j, -0.14414241+0.19370413j,
        -0.18510572+0.09015012j,  0.18887678-0.20250999j,
         0.02374265+0.21173011j, -0.03058101+0.09743701j,
        -0.0011431 +0.16597834j,  0.27659039-0.21551254j,
        -0.02495377-0.01257104j, -0.17457162+0.10333173j,
        -0.17643418-0.05441784j,  0.29649044+0.11901455j,
        -0.07878079-0.32799366j,  0.01821124+0.3039391j ],
       [-0.20576799-0.06968409j,  0.0740323 -0.12148095j,
        -0.11951468+0.08697454j, -0.20117197+0.069j     ,
         0.07987179-0.29709208j,  0.08735282+0.18202656j,
        -0.10865751+0.20239454j,  0.22146178-0.12153324j,
        -0.04641685+0.12215511j, -0.04035463+0.03191758j,
        -0.17598089-0.01289915j, -0.01468281-0.05960108j,
        -0.49896617+0.13751242j, -0.04311746+0.04246193j,
        -0.00099926+0.09646897j,  0.28284838-0.44498996j],
       [-0.24597721+0.06379617j, -0.01366898+0.35412976j,
        -0.2

### Prepare the cost function and the optimisation routine

In [7]:

Paulis = Variable(torch.zeros([4, 2, 2], dtype=torch.complex128), requires_grad=False)

# Use Pauli-Z as the Hermitian operator
Paulis[0] = torch.tensor([[1, 0], [0, -1]])
Paulis[1] = torch.tensor([[1, 0], [0, -1]])
Paulis[2] = torch.tensor([[1, 0], [0, -1]])
Paulis[3] = torch.tensor([[1, 0], [0, -1]])


## Required o/ps - Rows are the required output states
bloch_v=np.array([[0,0,1,1],
                  [0,1,0,1],
                  [1,0,1,0],
                  [1,1,0,0]])   
   
    

## Cost function
def cost_fn(params):
    f=0
    tt=np.zeros(4)
    for input_1 in range(4):
        a,b,c,d=circuit(params[4*input_1:(4*input_1)+4], Paulis[0],rand_sv_f[input_1])
        tt=[a,b,c,d]
        
        for k in range(4):
                     
            f+=torch.abs(tt[k] - bloch_v[input_1][k])
     
        
       
    return (f)



#set up the optimizer
opt = torch.optim.Adam([params], lr=0.001)

# number of steps in the optimization routine
steps = 1600

# the final stage of optimization isn't always the best, so we keep track of
# the best parameters along the way
best_cost = cost_fn(params)
best_params = np.zeros((nr_qubits*4, nr_layers, 4))

print("Cost after 0 steps is {:.4f}".format(cost_fn(params)))

# optimization begins
for n in range(steps):
    opt.zero_grad()
    loss = cost_fn(params)
    loss.backward()
    opt.step()

    # keeps track of best parameters
    if loss < best_cost:
        best_cost = loss
        best_params = params

    # Keep track of progress every 10 steps
    if n % 10 == 9 or n == steps - 1:
        print("Cost after {} steps is {:.4f}".format(n + 1, loss))




Cost after 0 steps is 8.8018


C:\Users\Bhargav\anaconda3\envs\Qiskit_1\lib\site-packages\torch\autograd\__init__.py:147: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  ..\aten\src\ATen\native\Copy.cpp:240.)
  Variable._execution_engine.run_backward(


Cost after 10 steps is 8.5847
Cost after 20 steps is 8.3463
Cost after 30 steps is 8.1117
Cost after 40 steps is 7.8951
Cost after 50 steps is 7.6846
Cost after 60 steps is 7.4825
Cost after 70 steps is 7.2966
Cost after 80 steps is 7.1365
Cost after 90 steps is 6.9885
Cost after 100 steps is 6.8483
Cost after 110 steps is 6.7303
Cost after 120 steps is 6.6130
Cost after 130 steps is 6.4998
Cost after 140 steps is 6.3880
Cost after 150 steps is 6.2770
Cost after 160 steps is 6.1683
Cost after 170 steps is 6.0593
Cost after 180 steps is 5.9491
Cost after 190 steps is 5.8390
Cost after 200 steps is 5.7473
Cost after 210 steps is 5.6619
Cost after 220 steps is 5.5788
Cost after 230 steps is 5.4970
Cost after 240 steps is 5.4139
Cost after 250 steps is 5.3333
Cost after 260 steps is 5.2539
Cost after 270 steps is 5.1749
Cost after 280 steps is 5.0974
Cost after 290 steps is 5.0212
Cost after 300 steps is 4.9440
Cost after 310 steps is 4.8703
Cost after 320 steps is 4.7946
Cost after 330 st

### Print the o/p results and compare with the desired o/p

In [8]:
output_bloch_v = [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,0]]
for input_1 in range(4):
    output_bloch_v[input_1] = circuit(best_params[4*input_1:(4*input_1)+4], Paulis[0],rand_sv_f[input_1])
        


# o/p after training
print("Output after training :")
print(output_bloch_v)

# desired o/p
print("Desired o/p :")
print(bloch_v)


Output after training :
[tensor([4.2476e-04, 6.5946e-04, 7.7024e-01, 8.0605e-01], dtype=torch.float64,
       grad_fn=<CatBackward>), tensor([-8.6857e-06,  9.1285e-01, -5.8013e-04,  6.4734e-01],
       dtype=torch.float64, grad_fn=<CatBackward>), tensor([ 7.1885e-01, -2.1761e-05,  6.4423e-01, -1.0951e-04],
       dtype=torch.float64, grad_fn=<CatBackward>), tensor([ 8.9348e-01,  7.1323e-01, -5.9763e-04, -1.0939e-04],
       dtype=torch.float64, grad_fn=<CatBackward>)]
Desired o/p :
[[0 0 1 1]
 [0 1 0 1]
 [1 0 1 0]
 [1 1 0 0]]
